In [ ]:
!rm -r 'IncrementalLeraningMLDL'
!git clone "https://github.com/wAnto97/IncrementalLeraningMLDL"
from IncrementalLeraningMLDL.src.CIFAR100_dataset import MyCIFAR100
from IncrementalLeraningMLDL.src.Utils import Utils
from IncrementalLeraningMLDL.src.MyNet import MyNet
from IncrementalLeraningMLDL.src.LfC import LfC
from IncrementalLeraningMLDL.src.Loss import Loss


import numpy as np
import sys
import copy
from torch.backends import cudnn
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from torch.utils.data import  DataLoader

# from google.colab import drive
# drive.mount('/content/gdrive')

rm: cannot remove 'IncrementalLeraningMLDL': No such file or directory
Cloning into 'IncrementalLeraningMLDL'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 1328 (delta 96), reused 99 (delta 53), pack-reused 1184
Receiving objects: 100% (1328/1328), 10.86 MiB | 4.75 MiB/s, done.
Resolving deltas: 100% (897/897), done.


**Loading data**

In [ ]:
train_transform = transforms.Compose([
                                      transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize( (0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))]) # Normalizes tensor with mean and standard deviation

# Define transforms for the evaluation phase
eval_transform = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize( (0.4914, 0.4822, 0.4465),(0.2023, 0.1994, 0.2010))])

training_set = MyCIFAR100('/content',train=True, n_groups=10, transform=train_transform, download=True,random_state = 653)
test_set = MyCIFAR100('/content',train=False, n_groups=10, transform=eval_transform, download=True,random_state = 653)

Files already downloaded and verified
Files already downloaded and verified


**Hyperparameters**

In [ ]:
DEVICE = 'cuda' # 'cuda' or 'cpu'  

BATCH_SIZE = 128      # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch strain_dataloaderize, learning rate should change by the same factor to have comparable results
LR = 0.2            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 1e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 70             # Total number of training epochs (iterations over dataset)
STEP_SIZE = [40,60]      # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1                 # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

CLASSES_PER_GROUP=10
NUM_GROUPS=10
NUM_EXEMPLARS=2000

**Utils functions**

In [ ]:
def validation(val_dataloader,net,lfc,conf_matrix=False,type_prediction = 'CNN'):
    net.train(False)
    running_corrects = 0
    y_pred = []
    all_labels = []
    for images, labels,_ in val_dataloader:

        images = images.to(DEVICE)
        labels = labels.to(DEVICE)

        # Forward Pass
        if type_prediction == 'NME':
          preds = lfc.predict(images,net)
        elif type_prediction == 'CNN':
          _,outputs = net(images)
          _, preds = torch.max(outputs.data, 1)
        # Update Corrects
        running_corrects += torch.sum(preds == labels.data).data.item()
        y_pred += list(map(lambda x : x.item(),preds))
        all_labels += list(labels)

        # Calculate Accuracy
    accuracy = running_corrects / float(len(val_dataloader.dataset))

    if(conf_matrix == True):
        all_labels = list(map(lambda label : label.item(),all_labels))
        return accuracy,confusion_matrix(y_pred,np.array(all_labels))

    return accuracy

In [ ]:
def test_loaders(train_dataloader):
    all_labels = []
    for images,labels,_ in train_dataloader:
      all_labels += list(map(lambda x: x.item(),labels))

    print(np.unique(all_labels,return_counts=True))

In [ ]:
def after_training(step,n_old_classes,train_dataloader,lfc,net,utils,training_set,type_prediction = 'CNN',type_reduction = 'random'):
  
  images_indices = utils.create_images_indices(train_dataloader,step)
  if type_prediction == 'NME':
    centroids = lfc.compute_centroids(net,training_set,images_indices,n_old_classes)

  if len(lfc.exemplar_set) > 0:
    print("Reducing the exemplar set..")
    lfc.reduce_exemplars(n_old_classes)
  
  print("Building the exemplar set...")
  if type_reduction == 'random':
    lfc.build_exemplars_random(images_indices,n_old_classes)
  elif type_reduction == 'herding':
    lfc.build_exemplars_herding(net,images_indices,n_old_classes)

  return 

**Main**

In [ ]:
myNet = MyNet(n_classes=CLASSES_PER_GROUP,type='cosine')
lfc = LfC(K=NUM_EXEMPLARS)
utils = Utils()
myLoss = Loss()
typeScheduler='multistep' # In this case it can be only set to multistep

#Creating dataloader for the first group of 10 classes
train_dataloader_exemplars,test_dataloader = utils.create_dataloaders_icarl(training_set,test_set,1,lfc.exemplar_set,BATCH_SIZE)

old_features=[]

for i in range(NUM_GROUPS):
    step=i+1
    print("STARTING Less Forget Constraint TRAINING WITH GROUP:\t",step)  
    n_old_classes = CLASSES_PER_GROUP*(step-1)
    
    if step > 1:
      myNet.update_network(myNet.net,CLASSES_PER_GROUP + n_old_classes,myNet.init_weights,type='cosine')
      train_dataloader_exemplars,test_dataloader = utils.create_dataloaders_icarl(training_set,test_set,step,lfc.exemplar_set,BATCH_SIZE)
      test_loaders(train_dataloader_exemplars)
    
    optimizer,scheduler = myNet.prepare_training(LR,MOMENTUM,WEIGHT_DECAY,STEP_SIZE,GAMMA,typeScheduler=typeScheduler,type='cosine')

    classification_losses = []
    distillation_losses = []

    myNet.net = myNet.net.to(DEVICE)
    cudnn.benchmark 

    for epoch in range(NUM_EPOCHS):
        running_correct_train = 0
        if typeScheduler == 'multistep':
          print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_last_lr()))
        elif typeScheduler == 'plateau':
          print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, optimizer.param_groups[0]['lr']))

        myNet.net.train() # Set Network to train mode
        current_step = 0
        for images, labels, _ in train_dataloader_exemplars:
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)

            #Set all gradients to zero
            optimizer.zero_grad() 

            #Computing output,features_map and creating the acyclic graph for updating the gradients
            new_features,outputs = myNet.net(images)

            #Computing predictions
            _, preds = torch.max(outputs.data,1)
            
            #Get features_map of the previous net
            if(step > 1):
                old_features = myNet.get_old_features_cosine(images,labels)

            #Computing loss
            if step>1:
              loss = myLoss.LfC_loss(old_features,new_features,outputs,labels,step,current_step,utils,myNet.net.eta(),lambda_base=10,n_classes=CLASSES_PER_GROUP,batch_size=BATCH_SIZE)
            else:
              loss = myLoss.LfC_loss(old_features,new_features,outputs,labels,step,current_step,utils,None,lambda_base=10,n_classes=CLASSES_PER_GROUP,batch_size=BATCH_SIZE)

            #Calculate correct predictions
            running_correct_train += torch.sum(preds == labels.data).data.item()

            #Accumulate gradients
            loss.backward() 

            # Update weights based on accumulated gradients
            optimizer.step() 

            current_step += 1

        #Calculate training accuracy
        train_accuracy = running_correct_train/len(train_dataloader_exemplars.dataset)
        print("Accuracy on the training :\t",train_accuracy)

        if typeScheduler == 'multistep':
            scheduler.step()
        elif typeScheduler == 'plateau':
            scheduler.step(val_loss)

    #Handling Exemplars
    exemplar_dataloader = DataLoader(training_set.get_group(step),batch_size=BATCH_SIZE,drop_last=False,num_workers=4)
    after_training(step,n_old_classes,exemplar_dataloader,lfc,myNet.net,utils,training_set)

    #Test
    test_accuracy,test_matrix = validation(test_dataloader,myNet.net,lfc,conf_matrix=True,type_prediction = 'CNN')
    print("Accuracy on the test :\t",test_accuracy)
    
    utils.writeOnFileMetrics('./LfCMetrics_CNN_5_5.json', step, [train_accuracy,None,test_accuracy,test_matrix.tolist()])
    # utils.writeOnFileLosses('./LfCLosses_CNN.json', step, [None,None])
    !cp   './LfCMetrics_CNN_5_5.json' './gdrive/My Drive/LfCMetrics_CNN_5_5.json'
    # !cp  './LfCLosses.json' './gdrive/My Drive/LfCLosses.json'

STARTING Less Forget Constraint TRAINING WITH GROUP:	 1
Starting epoch 1/70, LR = [0.02]
Accuracy on the training :	 0.2342
Starting epoch 2/70, LR = [0.02]
Accuracy on the training :	 0.394
Starting epoch 3/70, LR = [0.02]
Accuracy on the training :	 0.474
Starting epoch 4/70, LR = [0.02]
Accuracy on the training :	 0.5314
Starting epoch 5/70, LR = [0.02]
Accuracy on the training :	 0.5934
Starting epoch 6/70, LR = [0.02]
Accuracy on the training :	 0.626
Starting epoch 7/70, LR = [0.02]
Accuracy on the training :	 0.6574
Starting epoch 8/70, LR = [0.02]
Accuracy on the training :	 0.6878
Starting epoch 9/70, LR = [0.02]
Accuracy on the training :	 0.695
Starting epoch 10/70, LR = [0.02]
Accuracy on the training :	 0.7266
Starting epoch 11/70, LR = [0.02]
Accuracy on the training :	 0.7338
Starting epoch 12/70, LR = [0.02]
Accuracy on the training :	 0.7556
Starting epoch 13/70, LR = [0.02]
Accuracy on the training :	 0.7644
Starting epoch 14/70, LR = [0.02]
Accuracy on the training :